### 피어슨 상관계수 - 상관분석

In [29]:
import pandas as pd
from sklearn.datasets import load_diabetes

In [30]:
diabetes=load_diabetes()
data=pd.DataFrame(diabetes.data,columns=diabetes.feature_names)

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB


In [32]:
from scipy.stats import pearsonr

In [33]:
pearsonr(x=data['age'],y=data['bmi'])

PearsonRResult(statistic=0.1850846661465555, pvalue=9.076791865417418e-05)

- (상관계수,p-value)  
- 상관계수가 적어서 관계가 적음
- 유의수준 5% 귀무가설 기각  
- 상관계수는 작지만 유의미하다

In [34]:
data[['sex','bmi']].corr()

,sex,bmi
sex,1.000000,0.088161
bmi,0.088161,1.000000


### 샤피로 윌크 검정 - 정규성검정

In [35]:
import numpy as np

np.random.seed(2024)
x=np.random.random(10)

In [36]:
from scipy.stats import shapiro
shapiro(x)

ShapiroResult(statistic=0.8969663381576538, pvalue=0.20285381376743317)

- (통계량,p-value)형태 
- p-value가 커서 정규성을 만족하지 못함

## 모평균과 모분산 검정
### 단일표본 t검정
- ttest_1samp(list,popmean)
- defalut: popmean 과 다름

In [37]:
import numpy as np

kg=np.array([75.5,83.9,75.7,56.2,73.4,67.7,79.0,50.7,58.4,74.1,65.1,77.8,48.1,46.3])

np.mean(kg)

66.56428571428572

In [38]:
# 단일표본 t검정 실행
from scipy.stats import ttest_1samp
ttest_1samp(kg,70)

TtestResult(statistic=-1.0289933120202257, pvalue=0.3222484823978743, df=13)

- (t통계량값,p-value)
- p-value 5프로 이상이라 귀무가설을 기각을 못함 => 모집단 평균은 70이라 할수있따.

### 대응표본 t검정
- ttest_1samp(list,list)
- defalut : 두 표본의 평균의 차이가 0이 아님

In [39]:
import pandas as pd
female=np.array([50.7,58.4,74.1,65.1,77.8,48.1,46.3])
male=np.array([75.5,83.9,75.7,56.2,73.4,67.7,79.0])
diff=female-male
np.mean(diff)

-12.985714285714291

In [40]:
from scipy.stats import ttest_rel
ttest_rel(female,male)

TtestResult(statistic=-2.078446933064972, pvalue=0.08291274205610201, df=6)

- (t통계량,p-value)
- 0.83>5% 귀무가설 기각x -> 모평균이 같다고 할수있다

### 독립표본 t-검정
- ttest_ind(list,list)
- 등분산인지 여부
- 두 표본의 평균의 0이 아님

In [41]:
from scipy.stats import ttest_ind
ttest_ind(female,male)

Ttest_indResult(statistic=-2.2186641577772956, pvalue=0.046550122110569664)

- (t통계량,p-value)
- 0.46<5% -> 귀무가설기각 => 모평균은 서로 다르다

## 모분산 검정
### 단일표본 검정

In [42]:
import numpy as np
score=np.array([80.5,60.2,70,87,45,91,85])
var0=1100
var=np.var(score,ddof=1)    # 표본분산
print(var)

278.1033333333333


In [43]:
stat=(len(score)-1)*var/var0 # 검정통계량
print(stat)

1.5169272727272727


In [44]:
from scipy.stats import chi2
chi2.cdf(stat,len(score)-1)

0.041637780038918736

검정 결과가 5%보다 작으므로 귀무가설을 기각, 모분산이 1100보다 작다고 할수 없다

In [45]:
# 우측검정일 경우
1-chi2.cdf(stat,len(score)-1)

0.9583622199610813

In [46]:
# 양측검정일경우
2*chi2.cdf(stat,len(score)-1)

0.08327556007783747

### 분산비검정
- 두집단(a,b)에 대한 점수 데이터 임의로 생성하고 a집단의 모분산이 b집단의 모분산보다 작다고 할수있는 지를 검정해보자
- (가설은 a모분산< b모분산으로 가정,정규성 만족 가정)

In [47]:
# 두집단 (a,b)에 대한 점수데이터 임의로 생성
a=np.array([70,80,75,65,100,98])
b=np.array([20,100,50,94,28,80,95,30])

In [48]:
# 표본 분산 계산
var_a=np.var(a,ddof=1)
var_b=np.var(b,ddof=1)
print(var_a,var_b)

212.66666666666669 1138.4107142857142


In [49]:
# 검정통계량= 집단a의분산/집단b의분산(가설:a 모분산 <b 모분산)
stat=var_a/var_b
print(stat)

0.18681014154026346


In [50]:
# 자유도 계산
df1,df2=len(a)-1,len(b)-1
print(df1,df2)

5 7


In [51]:
# F 분포를 통해 직접 유의확률 계산
# 좌측검정이므로 Pr[F(자유도1,자유도2)<검정통계량]으로 계산
# 좌측 검정인 이유? 가설: a 모분산< b 모분산 <=> a모분산/b모분산<1
from scipy.stats import f
pval=f.cdf(stat,df1,df2)
print(pval)

0.04153943037562959


- 0.5보다 작으므로 귀무가설을 기각 => a집단의 모분산이 b집단의 모분산보다 작다고 할수있다.

### Bartlett 검정
- 이표본 이상에 대한 등분산검정법 => 정규성을 충족하는 데이터에 대해서 사용
- scipy.stats.bartlett(list,list,list.....)

In [52]:
# 임의로 세 그룹의 점수 데이터 생성
a=np.array([70,80,75,65,100,98])
b=np.array([20,100,50,94,28,80])
c=np.array([90,97,95,94,99,100])

In [53]:
# Bartlett 검정수행
from scipy.stats import bartlett
bartlett(a,b,c)

BartlettResult(statistic=15.6702722148674, pvalue=0.00039558846873743075)

p-value < 5% 귀무가설을 기각하므로 그룹간의 분산에 유의미한 차이가 있다고 할수 있다.

## Levene 검정
- 이표본 이상에 대한 등분산검정법 => 정규성을 충족하지 않는 데이터에 대해서 사용
- scipy.stats.levene(list,list,list,...)

In [54]:
from scipy.stats import levene
levene(a,b,c)

LeveneResult(statistic=14.365736704446384, pvalue=0.00032713621045500125)

- p-value < 5%  귀무가설 기각 그룹간 분산에 유의미한 차이가 있다고 할수있다

## 카이제곱 검정
### 적합성 검정
- scipy.stats.chisquare(관측도수,기대도수)

In [55]:
import numpy as np

color=['Black','Gold','Purple','Red','White']
counts=[423,304,274,205,294]
expected=300 #기대도수

In [56]:
# 적합성 검정 수행
from scipy.stats import chisquare
chisquare(f_obs=counts,f_exp=expected)

Power_divergenceResult(statistic=82.94, pvalue=4.14849046718008e-17)

- 카이제곱 통계량 82.94로 매우 크게 나타났고
- p-value < 5% 귀무가설을 기각하기 때문에 색상별로 동일한 비율로 생산되지 않고 있다

### 동질성 검정
- scipy.stats.chi2_contingency(RxC 교차표\)

- A회사의 성별에 따른 제품 선호도조사의 예를 보고 성별 간 선호도의 검증을 위한 동질성 검정을 수행하자.

In [57]:
import pandas as pd
from scipy.stats import chi2_contingency

In [58]:
obj={"Good":[400,350],"bad":[350,800]}
cross=pd.DataFrame(obj)
cross   # row간 합이 같아야하는데 책오류느낌..

,Good,bad
0,400,350
1,350,800


In [59]:
# 동질성 검정 수행
chi,p,df,expected =chi2_contingency(cross)
print(chi)  #카이제곱통계량 

98.66917336693975


In [60]:
print(p)    # p-값

2.984113573360021e-23


In [61]:
print(df)   # 자유도

1


In [62]:
print(expected) # 기대빈도

[[296.05263158 453.94736842]
 [453.94736842 696.05263158]]


p-value < 5% 귀무가설을 기각하기 떄문에 성별에 따라 선호도의 비율이 다르다고 할수있다.

### 독립성 검정
- scipy.stats.chi2_contingency(RxC 교차표)

- A회사의 출시된 제품의 한 달 간 판매량의 예이다. 이를 통해 성별과 색상이 서로 관련이 있는 지 알아보기 위해 독립성 검정을 수행해보자.

In [63]:
# 교차표에 맞게 데이터프레임 생성
obj={'Black':[1620,2380],
     'Gold':[385,615],
     'Purple':[778,1230],
     'Red':[394,610],
     'White':[800,180]}
cross=pd.DataFrame(obj)
cross # row간 합이다름

,Black,Gold,Purple,Red,White
0,1620,385,778,394,800
1,2380,615,1230,610,180


In [64]:
# 동질성 검정 수행
from scipy.stats import chi2_contingency
chi,p,df,expected=chi2_contingency(cross)
print(chi) # 카이제곱통계량

626.2830361414021


In [65]:
print(p)    # p-값

3.1731601033660286e-134


In [66]:
print(df) # 자유도

4


In [67]:
print(expected) # 기대빈도

[[1769.12811388  442.28202847  888.10231317  444.05115658  433.4363879 ]
 [2230.87188612  557.71797153 1119.89768683  559.94884342  546.5636121 ]]


- p-vale< 5% 귀무가설 기각 두변수는 종속적이다, 관련이 있다

## 비모수검정
- 분포가 확실치 않을때 하는 검정 방식
### 스피어만 상관계수 검정
- scipy.stats.spearmanr(list,list)

In [68]:
from scipy.stats import spearmanr
spearmanr(a=data['sex'],b=data['bmi'])

SignificanceResult(statistic=0.09807947297621517, pvalue=0.03929011358104615)

- corr 0.098 상관관계가 적음
- pvalue < 5% 귀무가설 기각 => 관계가 유의미하다고 할수있음

In [69]:
data[['sex','bmi']].corr(method='spearman')

,sex,bmi
sex,1.000000,0.098079
bmi,0.098079,1.000000


### 켄달의 타우 검정
- scipy.stats.kendalltau(list,list)
- default : 순위상관계수가 0이 아님

In [71]:
import numpy as np

x= np.array([5,4,3,6,1,2])
y=np.array([1,5,2,2,2,6])

In [72]:
# 켄달의 타우 검정 실시
from scipy.stats import kendalltau
kendalltau(x,y)

SignificanceResult(statistic=-0.29814239699997197, pvalue=0.4205962375999266)

- 상관계수 -0.29 상관계수가 거의 없음

### 윌콕슨의 부호순위 검정
- scipy.stats.wilcoxon(list,list)   => 하나만 입력할경우 1표본
- default : 분포가 0에대해 대칭적이지않음

In [73]:
from scipy.stats import wilcoxon

# 일표본 윌콕슨 부호순위 검정
# 몸무게 데이터의 임의 생성
kg=np.array([75.5,83.9,75.7,56.2,73.4,67.7,79.0,50.7,58.4,74.1,65.1,77.8,48.1,46.3])

In [75]:
# 두 샘플의 크기가 다를 수 있으므로 길이를 맞추기
wilcoxon(kg-70) #70으로 설정

WilcoxonResult(statistic=42.0, pvalue=0.5416259765625)

- pvalue > 5% 귀무가설기각x 모집단의 중위수가 70과 같다고 할수잇다

Q. '대응표본의 t-검정' 대신 이 표본 부호순위 검정을 수행해보자.

In [77]:
# 이표본 윌콕슨 부호순위 검정

# 남녀 몸무게 데이터를 임의로 생성
female=np.array([50.7,58.4,74.1,65.1,77.8,48.1,46.3])
male=np.array([75.5,83.9,75.7,56.2,73.4,67.7,79.0])

diff=female-male

In [78]:
wilcoxon(diff)

WilcoxonResult(statistic=5.0, pvalue=0.15625)

- p-value > 5% 두표본의 중앙값에 유의미한 차이가 없다

### 윌콕슨의 순위합 검정
- scipy.stats.ranksums(list,list)
- defalut : 분포중 하나가 다른분포보다 확률적으로 더큼

Q. '독립표본 t-검정' 대신 순위 랍 검정을 수행해보자.

In [80]:
from scipy.stats import ranksums
ranksums(female,male)

RanksumsResult(statistic=-1.8527420384998257, pvalue=0.06391934147515746)

- p-value > 5% => 귀무가설 기각x  => 두 표본의 중앙값에 유의미한 차이가 없다는 것

###  만 위트니 U검정
- scipy.stats.mannwhitneyu(list,list)
- defalut: 두분포가 동일하지 않음

In [83]:
# 만-위트니 U검정 수행
from scipy.stats import mannwhitneyu
mannwhitneyu(female,male)

MannwhitneyuResult(statistic=10.0, pvalue=0.07284382284382285)

- p-value > 5% => 귀무가설을 기각 x => 두표본의 중앙값에 유의미한 차이가 없다.

### 크루스칼- 왈리스 H검정
- scipy.stats.kruskal(**list)